# Connected component analysis (CCA)

## What is and object?

### Pixel Neighborhood

~~~
0 0 0 0 0 0 0 0
0 1 1 0 0 0 0 0
0 1 1 0 0 0 0 0
0 0 0 1 1 1 0 0
0 0 0 1 1 1 1 0
0 0 0 0 0 0 0 0
~~~

#### Orthogonal jumps:

* only jumps parallel to one of the axes allowed
* jump may only happen once for each axis

##### 1-jump neigborhood

```
- 1 -
1 x 1
- 1 -
```

-> result from above:
~~~
0 0 0 0 0 0 0 0
0 1 1 0 0 0 0 0
0 1 1 0 0 0 0 0
0 0 0 2 2 2 0 0
0 0 0 2 2 2 2 0
0 0 0 0 0 0 0 0
~~~


##### 2-jump neighborhood

```
2 1 2
1 x 1
2 1 2
```

-> result from above

~~~
0 0 0 0 0 0 0 0
0 1 1 0 0 0 0 0
0 1 1 0 0 0 0 0
0 0 0 1 1 1 0 0
0 0 0 1 1 1 1 0
0 0 0 0 0 0 0 0
~~~

### Exercise:

Consider the following "image":


How many connected objects are there with

1. 1-jump neighborhood

 * a. 1
 * b. 5
 * c. 2


2. 2-jump neighborhood

 * a. 2
 * b. 3
 * c. 5

## Connected Component Analysis

`skimage.measure.label`

* takes a binary image as an input (`False` is background, `True` foreground)
* CCA produces a new _labeled_ image with one integer number for each pixel (where pixels with the same number, belong to the same object)
* pixels neighborhood is specified in means of orthogonal jumps

### Exercise: How many objects are in that image

Now, it is your turn to practice.
Using the original `08-shapes.tif` image, add code to the following incomplete program, to count the number of objects in this image.
How many objects do you count manually? How does changing the `sigma` and `threshold` values influence the result?

In [ ]:
%load ../exercises/09-count-objects.py

## Morphometrics

### Exercise: Plot a histogram of the object area distribution

In [ ]:
%load ../exercises/09-count-objects.py

### Size filtering

### Exercise: print count of large objects only 

In [ ]:
%load ../exercises/09-count-objects.py

Keypoints:
- `skimage.measure.label` is used to generate objects.
- We use `skimage.measure.regionprops` to measure properties of labelled objects.
- Color objects according to feature values.